In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
recipes = pd.read_csv("drive/MyDrive/Recipes Generator/Data/recipes_data.csv")

# Filter the relevant columns
recipes_data = recipes[['NER', 'title']].dropna().iloc[:20000]
recipes_data = recipes_data.rename(columns={'NER': 'ingredients_list', 'title': 'recipe_title'})

In [ ]:
# Remove the square brackets and quotes
recipes_data['ingredients_list'] = recipes_data['ingredients_list'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace('"', '').strip())

In [ ]:
print(recipes_data)

                                        ingredients_list  \
0      bite size shredded rice biscuits, vanilla, bro...   
1      cream of mushroom soup, beef, sour cream, chic...   
2      frozen corn, pepper, cream cheese, garlic powd...   
3      chicken gravy, cream of mushroom soup, chicken...   
4      graham cracker crumbs, powdered sugar, peanut ...   
...                                                  ...   
19995  crackers, cream of mushroom soup, onion, water...   
19996  chili seasoning mix, sugar, tomatoes, salt, pe...   
19997  vanilla, shortening, brown sugar, eggs, soda, ...   
19998  vinegar, sugar, salt, pepper, regular oats, bu...   
19999  lemon juice, horseradish, grated cheese, crab ...   

                          recipe_title  
0                  No-Bake Nut Cookies  
1                Jewell Ball'S Chicken  
2                          Creamy Corn  
3                        Chicken Funny  
4                 Reeses Cups(Candy)    
...                              

In [ ]:
# Write each line to the file
with open("input.txt", "w", encoding="utf-8") as file:
    for _, row in recipes_data.iterrows():
        file.write(f"{row['ingredients_list']} - {row['recipe_title']}\n")

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Initialize and fit the tokenizer on the ingredients list and recipe titles
tokenizer = Tokenizer()
tokenizer.fit_on_texts(recipes_data['ingredients_list'].tolist() + recipes_data['recipe_title'].tolist())

In [ ]:
# Tokenize the ingredients list and recipe titles
recipes_data['ingredients_tokens'] = tokenizer.texts_to_sequences(recipes_data['ingredients_list'])
recipes_data['title_tokens'] = tokenizer.texts_to_sequences(recipes_data['recipe_title'])

In [ ]:
print(recipes_data)

                                        ingredients_list  \
0      bite size shredded rice biscuits, vanilla, bro...   
1      cream of mushroom soup, beef, sour cream, chic...   
2      frozen corn, pepper, cream cheese, garlic powd...   
3      chicken gravy, cream of mushroom soup, chicken...   
4      graham cracker crumbs, powdered sugar, peanut ...   
...                                                  ...   
19995  crackers, cream of mushroom soup, onion, water...   
19996  chili seasoning mix, sugar, tomatoes, salt, pe...   
19997  vanilla, shortening, brown sugar, eggs, soda, ...   
19998  vinegar, sugar, salt, pepper, regular oats, bu...   
19999  lemon juice, horseradish, grated cheese, crab ...   

                          recipe_title  \
0                  No-Bake Nut Cookies   
1                Jewell Ball'S Chicken   
2                          Creamy Corn   
3                        Chicken Funny   
4                 Reeses Cups(Candy)     
...                        

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length
MAX_LENGTH = 512

# Pad or truncate the tokenized sequences to a fixed length
recipes_data['ingredients_tokens'] = pad_sequences(
    recipes_data['ingredients_tokens'], maxlen=MAX_LENGTH, padding='post', truncating='post'
).tolist()

recipes_data['title_tokens'] = pad_sequences(
    recipes_data['title_tokens'], maxlen=MAX_LENGTH, padding='post', truncating='post'
).tolist()

In [ ]:
import torch

# Preprocessing: Ensure no None or empty lists exist
recipes_data['ingredients_tokens'] = [
    x if isinstance(x, list) else [] for x in recipes_data['ingredients_tokens']
]
recipes_data['title_tokens'] = [
    x if isinstance(x, list) else [] for x in recipes_data['title_tokens']
]

assert not recipes_data['ingredients_tokens'].apply(lambda x: x is None or len(x) == 0).any(), "There are still None or empty lists in ingredients_tokens!"
assert not recipes_data['title_tokens'].apply(lambda x: x is None or len(x) == 0).any(), "There are still None or empty lists in title_tokens!"

# Convert to tensors
input_ids = torch.tensor(recipes_data['ingredients_tokens'].tolist(), dtype=torch.long)
output_ids = torch.tensor(recipes_data['title_tokens'].tolist(), dtype=torch.long)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Prepare dataset for Trainer
data = {
    "input_ids": input_ids.tolist(),  # Convert tensors back to lists for Dataset
    "labels": output_ids.tolist(),    # Labels are identical to input_ids for GPT-2
}
dataset = Dataset.from_dict(data)

# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set up training arguments
training_args = TrainingArguments(
    output_dir='/content/results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/logs',
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="epoch",
)

# Prepare Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,  # Dataset with input_ids and labels
    eval_dataset=dataset,  # Using the same dataset for evaluation
)

# Train the model
trainer.train()

ModuleNotFoundError: No module named 'datasets'

In [ ]:
import json

# Save the tokenizer to a file
tokenizer_json = tokenizer.to_json()
with open('/content/drive/MyDrive/GPTresults/tokenizer.json', 'w') as file:
    file.write(tokenizer_json)
# Save model to Google Drive
model.save_pretrained('/content/drive/MyDrive/GPTresults')

In [ ]:
from transformers import AutoModelForCausalLM, GPT2Tokenizer
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import torch

# Load the tokenizer from the saved file
with open('/content/drive/MyDrive/GPTresults/tokenizer.json', 'r') as file:
    tokenizer_json = file.read()
tokenizer = tokenizer_from_json(tokenizer_json)

# Load the model using AutoModelForCausalLM (for SafeTensors support)
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/GPTresults', use_safetensors=True)

In [ ]:
!ls /content/drive/MyDrive/GPTresults

In [ ]:
tokenizer.texts_to_sequences(["wheat, sugar, eggs"])

In [ ]:
import torch

def generate_recipe(input_text, max_length=50, temperature=0.1, top_k=50, top_p=0.95):
    """
    Generate a recipe title based on input ingredients using the fine-tuned model.
    """
    # Set device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the device
    model.to(device)

    # Encode the input text and move to the same device as the model
    input_ids = torch.tensor(
        tokenizer.texts_to_sequences([input_text])[0], dtype=torch.long
    ).unsqueeze(0).to(device)  # Unsqueeze to add batch dimension

    # Create attention mask (1 for actual tokens, 0 for padding tokens)
    attention_mask = (input_ids != 0).long()  # Assuming 0 is the padding token

    # Generate output
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        pad_token_id=tokenizer.word_index.get("[PAD]", tokenizer.word_index.get("<pad>", 0)),  # Set pad token
    )

    # Decode the generated output (manually decode using tokenizer's reverse method)
    output_text = " ".join(tokenizer.sequences_to_texts(output.cpu().tolist()))

    return output_text

# Example usage
input_text = "ham, cheese, bread"
output_text = generate_recipe(input_text)
print(f"Ingredients:\n{input_text}\n")
print(f"Generated Title:\n{output_text}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Ingredients:
ham, cheese, bread

Generated Title:
ham cheese bread


# Old Work

In [ ]:
from datasets import load_dataset, Dataset
from transformers import GPT2Tokenizer

# Convert pandas DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(recipes_data)

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token # Set EOS token as padding token

def preprocess_full_recipe(example):
    if isinstance(example['ingredients_list'], str):
        ingredients_list = eval(example['ingredients_list'])
    else:
        ingredients_list = example['ingredients_list']

    ingredients_text = ', '.join(ingredients_list)
    recipe_title = example['recipe_title']
    print(len(recipe_title))
    print(len(ingredients_text))

    input_text = f"Ingredients: {ingredients_text} Title:"
    output_text = recipe_title

    # Tokenize inputs and targets
    input_encoding = tokenizer(
        input_text,
        truncation=True,
        padding='max_length',
        max_length=1000  # Adjust for your dataset
    )
    target_encoding = tokenizer(
        output_text,
        truncation=True,
        padding='max_length',
        max_length=1000  # Match max_length with inputs
    )

    # Return combined encoding
    return {
        'input_ids': input_encoding['input_ids'],
        'attention_mask': input_encoding['attention_mask'],
        'labels': target_encoding['input_ids'],
    }

# Tokenize the dataset
tokenized_data = hf_dataset.map(preprocess_full_recipe, batched=True)

# Split into training and testing datasets
split_data = tokenized_data.train_test_split(test_size=0.2)
train_dataset = split_data['train']
test_dataset = split_data['test']

In [ ]:
print(train_dataset[0])


In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

In [ ]:
test_example = "Ingredients: olive oil, garlic, pasta, tomatoes Title:"
inputs = tokenizer(test_example, return_tensors='pt', truncation=True, padding=True, max_length=256)
inputs = {key: val.to(model.device) for key, val in inputs.items()}
outputs = model.generate(**inputs, max_length=50, num_return_sequences=1, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))